In [4]:
import QuantLib as ql
import pandas as pd
import math

# =========================
# Setup
# =========================

eval_date = ql.Date(4, 1, 2022)
ql.Settings.instance().evaluationDate = eval_date

# Proper flat curve at 3%
yts = ql.FlatForward(eval_date, 0.03, ql.Actual360())
ts_handle = ql.YieldTermStructureHandle(yts)

# Create Ibor index and set fixing
index = ql.Euribor3M(ts_handle)
calendar = ql.TARGET()
fixing_date = calendar.advance(eval_date, -2, ql.Days)
index.addFixing(fixing_date, 0.03)

# Create schedule
start_date = eval_date
end_date = start_date + ql.Period(2, ql.Years)
schedule = ql.Schedule(
    start_date,
    end_date,
    ql.Period(3, ql.Months),
    calendar,
    ql.ModifiedFollowing,
    ql.ModifiedFollowing,
    ql.DateGeneration.Backward,
    False,
)

# Cap and floor
notional = 1_000_000
strike = 0.03  # ATM
cap = ql.Cap(ql.IborLeg([notional], schedule, index), [strike])
floor = ql.Floor(ql.IborLeg([notional], schedule, index), [strike])

# =========================
# Volatility conversion
# =========================

black_vol = 0.20
forward_rate = 0.03

# Bachelier vol = Black vol × Forward rate
# This is the Rebonato approximation
bachelier_vol = black_vol * forward_rate

print("="*70)
print("Cap and Floor Pricing - Four Methods")
print("="*70)
print(f"\nVolatility Conversion:")
print(f"  Black vol (lognormal):     {black_vol:.4f} (20%)")
print(f"  Forward rate:              {forward_rate:.4f} (3%)")
print(f"  Bachelier vol (normal):    {bachelier_vol:.6f} ({bachelier_vol*10000:.2f} bps)")

results = {}

# =========================
# 1) BlackCapFloorEngine
# =========================

vol_black_quote = ql.QuoteHandle(ql.SimpleQuote(black_vol))
engine_black = ql.BlackCapFloorEngine(ts_handle, vol_black_quote)
cap.setPricingEngine(engine_black)
floor.setPricingEngine(engine_black)

cap_black = cap.NPV()
floor_black = floor.NPV()
results['Black'] = (cap_black, floor_black)

print("\n1) BlackCapFloorEngine (lognormal, vol=20%):")
print(f"   Cap:  {cap_black:,.2f}")
print(f"   Floor: {floor_black:,.2f}")

# =========================
# 2) BachelierCapFloorEngine (with correct vol)
# =========================

vol_bachelier_quote = ql.QuoteHandle(ql.SimpleQuote(bachelier_vol))
engine_bachelier = ql.BachelierCapFloorEngine(ts_handle, vol_bachelier_quote)
cap.setPricingEngine(engine_bachelier)
floor.setPricingEngine(engine_bachelier)

cap_bachelier = cap.NPV()
floor_bachelier = floor.NPV()
results['Bachelier'] = (cap_bachelier, floor_bachelier)

print("\n2) BachelierCapFloorEngine (normal, vol converted):")
print(f"   Cap:  {cap_bachelier:,.2f}")
print(f"   Floor: {floor_bachelier:,.2f}")

# =========================
# 3) AnalyticCapFloorEngine (Hull-White)
# =========================

hw_model = ql.HullWhite(ts_handle, a=0.03, sigma=0.01)
engine_analytic = ql.AnalyticCapFloorEngine(hw_model, ts_handle)

cap.setPricingEngine(engine_analytic)
floor.setPricingEngine(engine_analytic)

cap_analytic = cap.NPV()
floor_analytic = floor.NPV()
results['AnalyticHW'] = (cap_analytic, floor_analytic)

print("\n3) AnalyticCapFloorEngine (Hull-White model):")
print(f"   Cap:  {cap_analytic:,.2f}")
print(f"   Floor: {floor_analytic:,.2f}")

# =========================
# 4) TreeCapFloorEngine (Hull-White)
# =========================

hw_tree = ql.HullWhite(ts_handle, a=0.03, sigma=0.01)
engine_tree = ql.TreeCapFloorEngine(hw_tree, 60, ts_handle)

cap.setPricingEngine(engine_tree)
floor.setPricingEngine(engine_tree)

cap_tree = cap.NPV()
floor_tree = floor.NPV()
results['TreeHW'] = (cap_tree, floor_tree)

print("\n4) TreeCapFloorEngine (Hull-White tree, 60 steps):")
print(f"   Cap:  {cap_tree:,.2f}")
print(f"   Floor: {floor_tree:,.2f}")



Cap and Floor Pricing - Four Methods

Volatility Conversion:
  Black vol (lognormal):     0.2000 (20%)
  Forward rate:              0.0300 (3%)
  Bachelier vol (normal):    0.006000 (60.00 bps)

1) BlackCapFloorEngine (lognormal, vol=20%):
   Cap:  3,998.05
   Floor: 3,801.92

2) BachelierCapFloorEngine (normal, vol converted):
   Cap:  3,997.91
   Floor: 3,801.78

3) AnalyticCapFloorEngine (Hull-White model):
   Cap:  6,594.24
   Floor: 6,398.11

4) TreeCapFloorEngine (Hull-White tree, 60 steps):
   Cap:  6,575.86
   Floor: 6,351.74
